In [1]:
import torch
from torch import Tensor
from torch import nn 
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
from utility import *

np.random.seed(seed=7)

In [2]:
import dlc_bci

train_input_100 , train_target_100 = dlc_bci.load(root = './data_bci_100Hz', download = False)
test_input_100 , test_target_100 = dlc_bci.load(root = './data_bci_100Hz', download = False, train = False)

train_input_1000 , train_target_1000 = dlc_bci.load(root = './data_bci_1000Hz', download = False, one_khz = True)
test_input_1000 , test_target_1000 = dlc_bci.load(root = './data_bci_1000Hz', download = False, train = False, one_khz = True)

print("Train input 100 Hz: {:d}x{:d}x{:d}".format(*(s for s in train_input_100.size())))
print("Train target 100 Hz: {:d}".format(*(s for s in train_target_100.size())))
print("Test input 100 Hz: {:d}x{:d}x{:d}".format(*(s for s in test_input_100.size())))
print("Test target 100 Hz: {:d}".format(*(s for s in test_target_100.size())))
print("")
print("Train input 1000 Hz: {:d}x{:d}x{:d}".format(*(s for s in train_input_1000.size())))
print("Train target 1000 Hz: {:d}".format(*(s for s in train_target_1000.size())))
print("Test input 1000 Hz: {:d}x{:d}x{:d}".format(*(s for s in test_input_1000.size())))
print("Test target 1000 Hz: {:d}".format(*(s for s in test_target_1000.size())))

Train input 100 Hz: 316x28x50
Train target 100 Hz: 316
Test input 100 Hz: 100x28x50
Test target 100 Hz: 100

Train input 1000 Hz: 316x28x500
Train target 1000 Hz: 316
Test input 1000 Hz: 100x28x500
Test target 1000 Hz: 100


# Network definition

This conv2d network is more inspired from a classic conv2d net that is used for image recognition 

In [3]:
from models import *

In [4]:
def compute_nb_errors(model, data_input, data_target, batch_size):
    nb_errors = 0
    Ndata = len(data_input[:, 0, 0, 0])
    model.eval()
    print('data_target', data_target.shape)
    print('data_input', data_input.shape)
    
    for b_start in range(0, Ndata, batch_size):
        bsize_eff = batch_size - max(0, b_start+batch_size-Ndata)  # boundary case
        batch_output = model.forward(data_input.narrow(0, b_start, bsize_eff))  # is Variable if data_input is Variable
        if len(list(batch_output.size()))>1 and batch_output.size(1) > 1:
            # as many ouputs as there are classes => select maximum output
            nb_err_batch = (batch_output.max(1)[1] != data_target.narrow(0, b_start, bsize_eff)).long().sum()
            # overflow problem if conversion to Long Int not performed, treated as short 1-byte int otherwise!!
        else:
            # output is a scalar in [0, 1]
            nb_err_batch = batch_output.round().sub(data_target.narrow(0, b_start, bsize_eff)).sign().abs().sum()
        
        nb_errors += nb_err_batch
    if isinstance(nb_errors, Variable):
        nb_errors = nb_errors.data[0]
    return nb_errors

# Preprocessing and Data Augmentation

In [5]:
import numpy as np

preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target, kept_channels_idx = preprocessing_train(train_input_1000, train_target_1000, subsampling_frequency='125Hz', window=True,denoize=False, addGaussianNoise=False, reduceChannels=False, cutEnd = False)
preprocessed_input_test, preprocessed_input_test_target  = preprocessing_test(test_input_1000, test_target_1000, kept_channels_idx, subsampling_frequency='125Hz', window=True,denoize = False, reduceChannels=False, cutEnd=False)

#Remove Noise
#preprocessed_input_train = denoisedSignals(preprocessed_input_train)
#preprocessed_input_validation = denoisedSignals(preprocessed_input_validation)
#preprocessed_input_test = denoisedSignals(preprocessed_input_test)
#add random noise
#preprocessed_input_train = whiteNoise(preprocessed_input_train)
#preprocessed_input_validation = whiteNoise(preprocessed_input_validation)
#preprocessed_input_test = whiteNoise(preprocessed_input_test)

print('train', preprocessed_input_train.shape)
print('test', preprocessed_input_test.shape)
print('validation', preprocessed_input_validation.shape)

labels_train = torch.from_numpy(preprocessed_input_train_target)
labels_test = torch.from_numpy(preprocessed_input_test_target)
labels_validation = torch.from_numpy(preprocessed_input_validation_target)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train).float()
preprocessed_input_test = torch.from_numpy(preprocessed_input_test).float()
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation).float()

Ntrain = len(preprocessed_input_train[:,0,0])
Ntest = len(preprocessed_input_test[:,0,0])
Nvalidation = len(preprocessed_input_validation[:,0,0])

Nchannels = len(preprocessed_input_train[0,:,0])
Nsamples_100 = preprocessed_input_train.size(-1)

print('Ntrain = ', Ntrain)
print('Ntest = ', Ntest)
print('Nvalidation = ', Nvalidation)

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, Nchannels, Nsamples_100))
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, Nchannels, Nsamples_100), requires_grad=False)
test_input = Variable(preprocessed_input_test.contiguous().view(Ntest, 1, Nchannels, Nsamples_100), requires_grad=False)

train (18300, 28, 8)
test (6100, 28, 8)
validation (976, 28, 8)
Ntrain =  18300
Ntest =  6100
Nvalidation =  976


# Network training and testing
Non-linearity: elu  


|criterion | optimizer | lr  | momentum | batch size | Nepochs | Train acc. | Test acc.|
|----------|-----------|-----|----------|------------|---------|------------|----------|
| BCE  | Adam  |1e-1 | def. | 15 | 150 | 86.4 | 61.4 | 
| BCE  | Adam  |1e-1 | def. | 20 | 150 | 99.8 | 79.5 | 
| BCE  | SGD   | 1e-2 | 0.85 | 20 | 150 | 98.9  | 61.5 | 
| CE   | Adam  | 1e-2 | def. | 20 | 150 | 98.4  |  70.5 | 
| CE   | SGD   | 1e-2 | 0.85 | 20 | 150 | 99.1 | 75.1 |


Non-linearity: ReLU

In [ ]:
import torch.optim as optim
import math

# Train network 
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(labels_train)  # keep long tensors
    validation_target = Variable(labels_validation, requires_grad=False) # convert to float
    test_target = Variable(labels_test, requires_grad=False)
    Noutputs = 2
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(labels_train)  # keep long tensors
    validation_target = Variable(labels_validation, requires_grad=False) # convert to float
    test_target = Variable(labels_test, requires_grad=False )
    Noutputs = 2
    
else:
    train_target = Variable(labels_train.float()) # convert to float
    validation_target = Variable(labels_validation.float(), requires_grad=False) # convert to float
    test_target = Variable(labels_test.float(), requires_grad=False)
    Noutputs = 1

batch_size = 15
Nbatches = int(math.ceil(Ntrain/batch_size))
Nepochs = 50
seeds = list(range(50))
#seeds = [14,20,21,22,27,46,44,37,31,28]
Nrep = len(seeds)

train_errors = torch.Tensor(Nrep, Nepochs).zero_()
test_errors = torch.Tensor(Nrep, Nepochs).zero_()
validation_errors = torch.Tensor(Nrep, Nepochs).zero_()
ep_loss = torch.Tensor(Nrep, Nepochs).zero_()

for i_rep in range(Nrep):
    
    print('Repetition', seeds[i_rep])
    torch.manual_seed(seeds[i_rep])
    #model = conv2DNet_1(Noutputs) #from classic knowledge of image segmentation  
    #model = conv2DNet_2(Nchannels, Nsamples_100, Noutputs) #from litterature 
    #model = conv2DNet_3(Noutputs) #from PdM
    #model = conv2DNet_4(Noutputs) #from PdM
    #model = conv2DNet_5(Noutputs) #from PdM
    model = conv2DNet_6(Noutputs)
    
    #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.50)
    optimizer = optim.Adam(model.parameters())
    #optimizer = optim.Adagrad(model.parameters())
    #optimizer = optim.Adamax(model.parameters())
    #optimizer = optim.ASGD(model.parameters())
    #optimizer = optim.RMSprop(model.parameters())
    #optimizer = optim.Rprop(model.parameters())
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

    for i_ep in range(Nepochs):
        for b_start in range(0, Ntrain, batch_size):
            bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
            model.train()
            model.zero_grad()
            output = model(train_input.narrow(0, b_start, bsize_eff))
            batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))            
            ep_loss[i_rep, i_ep] += batch_loss.data[0]
            batch_loss.backward()
            optimizer.step()
        
        scheduler.step(ep_loss[i_rep, i_ep])
        
        nb_train_errs = compute_nb_errors(model, train_input, train_target, batch_size)
        nb_validation_errs = compute_nb_errors(model, validation_input, validation_target, batch_size)
        nb_test_errs = compute_nb_errors(model, test_input, test_target, batch_size)
        
        print("Epoch Number : ", i_ep)
        print("\t Training accuracy: ", (100*(Ntrain-nb_train_errs)/Ntrain))
        print("\t Validation accuracy ",(100*(Nvalidation-nb_validation_errs)/Nvalidation)) 
        print("\t Test accuracy ",(100*(Ntest-nb_test_errs)/Ntest))
        
        print("\t Epoch Loss ", ep_loss[i_rep, i_ep])
        
        train_errors[i_rep, i_ep] = nb_train_errs
        test_errors[i_rep, i_ep] = nb_test_errs
        validation_errors[i_rep, i_ep] = nb_validation_errs

Repetition 0
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  0
	 Training accuracy:  63.28415300546448
	 Validation accuracy  52.76639344262295
	 Test accuracy  56.90163934426229
	 Epoch Loss  771.6889038085938
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  1
	 Training accuracy:  72.26229508196721
	 Validation accuracy  60.65573770491803
	 Test accuracy  56.704918032786885
	 Epoch Loss  668.82421875
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  2
	 

Epoch Number :  21
	 Training accuracy:  90.02732240437159
	 Validation accuracy  55.635245901639344
	 Test accuracy  46.73770491803279
	 Epoch Loss  31.190170288085938
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  22
	 Training accuracy:  88.06010928961749
	 Validation accuracy  55.122950819672134
	 Test accuracy  47.967213114754095
	 Epoch Loss  40.34190368652344
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  23
	 Training accuracy:  90.08743169398907
	 Validation accuracy  50.10245901639344
	 Test accuracy  46.18032786885246
	 Epoch Loss  60.49391555786133
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 

data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  43
	 Training accuracy:  91.60109289617486
	 Validation accuracy  49.385245901639344
	 Test accuracy  47.21311475409836
	 Epoch Loss  18.51148223876953
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  44
	 Training accuracy:  91.55737704918033
	 Validation accuracy  50.0
	 Test accuracy  47.67213114754098
	 Epoch Loss  15.793169021606445
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  45
	 Training accuracy:  90.51912568306011
	 Validation accuracy  48.05327868852459
	 Test accu

data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  15
	 Training accuracy:  88.33879781420765
	 Validation accuracy  49.385245901639344
	 Test accuracy  51.0655737704918
	 Epoch Loss  64.1166763305664
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  16
	 Training accuracy:  87.96174863387978
	 Validation accuracy  45.1844262295082
	 Test accuracy  46.704918032786885
	 Epoch Loss  65.05835723876953
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  17
	 Trainin

data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  37
	 Training accuracy:  89.11475409836065
	 Validation accuracy  48.97540983606557
	 Test accuracy  44.42622950819672
	 Epoch Loss  36.64767837524414
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  38
	 Training accuracy:  89.43169398907104
	 Validation accuracy  43.75
	 Test accuracy  47.65573770491803
	 Epoch Loss  29.87310218811035
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  39
	 Training accuracy:

...

data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  33
	 Training accuracy:  90.35519125683061
	 Validation accuracy  46.51639344262295
	 Test accuracy  49.540983606557376
	 Epoch Loss  47.1661376953125
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  34
	 Training accuracy:  91.6775956284153
	 Validation accuracy  46.92622950819672
	 Test accuracy  49.40983606557377
	 Epoch Loss  27.811744689941406
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  35
	 Traini

data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  5
	 Training accuracy:  84.77049180327869
	 Validation accuracy  59.01639344262295
	 Test accuracy  50.42622950819672
	 Epoch Loss  220.93563842773438
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  6
	 Training accuracy:  82.5464480874317
	 Validation accuracy  58.299180327868854
	 Test accuracy  52.704918032786885
	 Epoch Loss  164.17095947265625
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  7
	 Traini

Epoch Number :  26
	 Training accuracy:  89.9672131147541
	 Validation accuracy  55.22540983606557
	 Test accuracy  47.34426229508197
	 Epoch Loss  41.09007263183594
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  27
	 Training accuracy:  87.65573770491804
	 Validation accuracy  54.09836065573771
	 Test accuracy  49.59016393442623
	 Epoch Loss  29.822444915771484
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  28
	 Training accuracy:  90.59562841530055
	 Validation accuracy  47.950819672131146
	 Test accuracy  45.90163934426229
	 Epoch Loss  40.655242919921875
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8]

data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  48
	 Training accuracy:  92.79781420765028
	 Validation accuracy  47.64344262295082
	 Test accuracy  46.967213114754095
	 Epoch Loss  26.206159591674805
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  49
	 Training accuracy:  91.14207650273224
	 Validation accuracy  54.09836065573771
	 Test accuracy  47.32786885245902
	 Epoch Loss  13.52952766418457
Repetition 33
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  0
	 Training accuracy:  64.32786885245902
	 Validation accuracy  52.

data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  20
	 Training accuracy:  90.68306010928961
	 Validation accuracy  47.540983606557376
	 Test accuracy  48.295081967213115
	 Epoch Loss  44.292537689208984
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  21
	 Training accuracy:  88.49726775956285
	 Validation accuracy  43.23770491803279
	 Test accuracy  51.81967213114754
	 Epoch Loss  68.80936431884766
data_target torch.Size([18300])
data_input torch.Size([18300, 1, 28, 8])
data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
data_target torch.Size([6100])
data_input torch.Size([6100, 1, 28, 8])
Epoch Number :  22
	 Tra

In [ ]:
print(train_errors.shape)

for i_rep in range(Nrep):
    print(i_rep)
    plt.plot(np.array(train_errors[i_rep, :]))
    plt.plot(np.array(validation_errors[i_rep, :]))
    plt.plot(np.array(test_errors[i_rep, :]))
    plt.show()

In [ ]:
for i_rep in range(Nrep):
    print(i_rep)
    plt.plot(100*(Ntrain-np.array(train_errors[i_rep, :]))/Ntrain)
    plt.plot(100*(Nvalidation-np.array(validation_errors[i_rep, :]))/Nvalidation)
    plt.plot(100*(Ntest-np.array(test_errors[i_rep, :]))/Ntest)

    plt.show()

In [ ]:
train_accuracy = 100*(Ntrain-np.array(train_errors))/Ntrain
val_accuracy = 100*(Nvalidation-np.array(validation_errors))/Nvalidation
test_accuracy = 100*(Ntest-np.array(test_errors))/Ntest

stddev_train_errors = np.std(train_accuracy, axis=0)
stddev_val_errors = np.std(val_accuracy, axis=0)
stddev_test_errors = np.std(test_accuracy, axis=0)

mean_train_errors = np.mean(train_accuracy, axis=0)
mean_val_errors = np.mean(val_accuracy, axis=0)
mean_test_errors = np.mean(test_accuracy, axis=0)

epoch = list(range(75))
print(stddev_train_errors.shape)

plt.errorbar(epoch, mean_train_errors, stddev_train_errors)
plt.errorbar(epoch, mean_val_errors, stddev_val_errors)
plt.errorbar(epoch, mean_test_errors, stddev_test_errors)
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy in %')
plt.legend(['train', 'validation', 'test'])